In [ ]:
# Install the required libraries (from pip_requirements.txt)
%pip install -r pip_requirements.txt

!python -m spacy download en_core_web_sm

In [ ]:
# Modules to import
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
from importlib import reload

In [ ]:
paths = ['src/data','src/models','src/scripts','src/utils']
for path in paths:
    sys.path.append(path)

# NLP Utilization

These codes are the ones we've used to calculate the 'US influence score' for each country

### Exemple of the principle with one summary

In [ ]:
#Example of the principle of our code applied to a summary

from transformers import pipeline

# Here we use the zero-shot-classification pipeline
nlp = pipeline("zero-shot-classification")

# movie summary example
resume = "In Los Angeles, California Fletcher Reede  is a career-focused lawyer and divorced father. He loves spending time with his young son Max ([[Justin Cooper ; they often play a game where Fletcher makes his hand into the Claw and pretends to chase Max with it. But Fletcher has a habit of giving precedence to his job, breaking promises to Max and his ex-wife Audrey , and then lying about the reasons. Fletcher's compulsive lying has also built him a reputation as one of the best defense lawyers in the state of California as he is climbing the ladder in the firm for which he works. Ultimately, Fletcher misses his son's fifth birthday party because he is having sex with his boss Miranda , in the hopes of making partner. Max makes a birthday wish that his father would be unable to tell a lie for an entire day &mdash; a wish that immediately becomes true. Fletcher soon discovers, through a series of embarrassing incidents &mdash; such as when he tells Miranda that he has had better sex than he just did with her &mdash; that he is unable to lie, mislead, or even withhold a true answer. These incidents are inconvenient, as he is fighting a divorce case in court which, should he win, could be a huge boost to his career. His client is the self-centered, money grabbing Samantha Cole . His main witness is willing to commit perjury to win, but Fletcher discovers that he cannot even ask a question if he knows the answer will be a lie; during the case he even objects to himself when he tries to lie to get the desired information. Meanwhile, Audrey is planning to move to Boston with her new boyfriend Jerry , and decides that Max is going with them so that she can protect him from getting hurt by Fletcher. Fletcher tries desperately to delay the case, even beating himself up, but he cannot conceal that he is able to continue, so the judge  insists that he does. Finally he realizes that his client had lied about her age and therefore had signed her prenuptial agreement while a minor, rendering it invalid. This allows him to win his case truthfully, but the repercussions become a catalyst to his understanding of what he himself is about to lose. Samantha who only cared about her ex-husband's money, takes custody of her children purely so her ex-husband would be forced to continue paying her for child care and literally pulls the children out of the arms of their responsible and caring father. Fletcher then has a crisis of conscience and shouts at the judge to reverse the decision, and is taken to jail for contempt of court. Audrey refuses to pay his bail, which is eventually paid by his secretary Greta , who forgives him for his earlier rude truth-tellings after hearing he went all noble in front of their firm's senior partner. Now recognizing his son Max as his highest priority, Fletcher struggles to keep him. He hurries to the airport, but Audrey and Max's plane has already left the terminal. Desperate, he hijacks a mobile stairway and pursues the plane onto the runway. The plane finally stops, but Fletcher is injured. On his way to the hospital, he vows to his son to spend more time with him and never hurt him again. Even though the 24 hours of truth are up, Max knows he means it, and Audrey decides not to move to Boston with Jerry after all. One year later, Fletcher and Audrey are celebrating Max's birthday. The lights go off when Max blows out his birthday candles. When they go back on, Fletcher and Audrey are kissing. Fletcher asks Max if he wished for his mom and his dad to get back together and Max replies No, I wished for roller blades! The family seemingly returns to normal as Fletcher chases Audrey and Max around the house with the Claw."

# Here we define the label (Unites States) that we want to use to classify the text
labels = ["United States"]

result = nlp(resume, candidate_labels=labels)

# The result is a dataframe with the labels and their respective scores
df_result = pd.DataFrame(list(zip(result['labels'], result['scores'])), columns=['Label', 'Score'])

df_result.head()


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


,Label,Score
0,United States,0.299839


## Code to calculate the US Influence Score on the differents movies

Our aim with this code is to use NLP on the summaries to determine an influence score for the US for each movie. We will store the results, along with the score for each movie, in a CSV file called "scores.csv," which will be used in our analysis.

In [8]:
# Import dataloader function
from dataLoader import loadDataframe

In [ ]:
# Load data
path_to_directory = 'data/cleanData/'
df_movies = loadDataframe('movies', path_to_directory)
df_characters = loadDataframe('characters', path_to_directory)
df_clusters = loadDataframe('clusters', path_to_directory)
df_summaries = loadDataframe('summaries', path_to_directory)

df_movies_merge = pd.merge(df_summaries, df_movies, how='inner')
# Remove movies from the United States (useless for the US influence score and enable to treat half of the movies)
df_movies_merge = df_movies_merge[~df_movies_merge['countries'].apply(lambda x: 'United States of America' in x)]
df_movies_merge = df_movies_merge[['wiki_id','summary','countries']]

The following function calculates the match score between the summary and the label "United States" for each movie and saves it in the "scores.csv" file. I can choose the batch size each time, as even after removing the United States, the size of my DataFrame exceeds 21,000 rows and I can't process everything at once.

In [ ]:
import os
import pandas as pd
from transformers import pipeline

# Load the zero-shot classification pipeline
nlp = pipeline("zero-shot-classification")

# Example function to calculate the score
def compute_score(row, label):
    """
    Calculate the match score between the summary and our label, United States.
    
    Args:
    - row: A row from the DataFrame containing at least 'summary'.
    - label: United States.
    
    Returns:
    - Match score between the text and the United States.
    """
    result = nlp(row['summary'], candidate_labels=[label])
    return result['scores'][0]  # The score for the first label (single label)

# Function to process the DataFrame in chunks
def process_in_chunks(df, label, chunk_size=100, output_file='scores.csv'):
    """
    Process the DataFrame in several chunks and save the results.
    
    
    """
    # If the file already exists, load it to avoid overwriting already calculated results
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        start_index = len(existing_df)  # Resume from the last calculated row
        print(f"Resuming from row {start_index}")
    else:
        existing_df = pd.DataFrame()
        start_index = 0  # If the file doesn't exist, start from 0
    
    # Split the DataFrame into chunks
    for i in range(start_index, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        # Calculate scores for this chunk
        chunk.loc[:, 'score'] = chunk.apply(lambda row: compute_score(row, label), axis=1)
        
        # Use pd.concat() to add the chunk to the existing DataFrame
        existing_df = pd.concat([existing_df, chunk], ignore_index=True)
        
        # Save this chunk to the output file
        existing_df.to_csv(output_file, index=False)
        print(f"Progress: Processing rows {i} to {i+chunk_size-1}")



# Process in chunks (for example, 1000 rows at a time)
process_in_chunks(df_movies_merge, label='United States', chunk_size=1000, output_file='scores.csv')


## Code to determine the origin of the characters

This is the code that we've used to compute the "Best_Country" for each character corresponding to the origin of the character

Our goal with this code is to calculate, using the same NLP method as before, an attribution score for each character to each country. The result will be a table containing the name of each character that appears in multiple films, the country with the highest score, and the associated attribution score. This table will be named "character_countries.csv."

In [13]:
from scriptculture import process_data_character          # Import the function that processes the data that we'll use

df_character_influence = process_data_character()

The following function calculates a matching score for each character with the different countries (here, for computational efficiency, we only consider countries that have produced film where the character appears). Similarly, the function allows you to choose a batch size to process the data in multiple steps.

In [ ]:
import pandas as pd
from transformers import pipeline
import os

# Load the zero-shot classification pipeline
nlp = pipeline("zero-shot-classification")

# Function to compute the country with the highest score
def compute_best_country(row):
    """
    Computes the country with the highest score for a given character.
    
    Args:
    - row: A row from the DataFrame containing the 'Character' column.
    
    Returns:
    - A dictionary containing the country with the highest score and the associated score.
    """
    labels = row['all_countries']   #The labels correspond to the countries that produced the films in which the character appears.
    if not labels:  # If the list is empty, return default values
        return None, None
    result = nlp(row['Character'], candidate_labels=labels)
    best_country = result['labels'][0]  # Country with the highest score
    best_score = result['scores'][0]    # Associated score
    return best_country, best_score

# Function to process in chunks and save results
def process_in_chunks_with_best_country(df, chunk_size, output_file='character_countries.csv'):
    """
    Processes a DataFrame in chunks and calculates the country with the highest score for each character.
    
    Args:
    - df: The DataFrame containing the 'Character' column.
    - chunk_size: The number of rows to process at a time.
    - output_file: The file to save the results.
    """
    # If the file exists, load existing data
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        start_index = len(existing_df)  # Resume from the last processed row
        print(f"Resuming from row {start_index}")
    else:
        existing_df = pd.DataFrame(columns=['Character', 'Best_Country', 'Best_Score'])
        start_index = 0
    
    # Process the DataFrame in chunks
    for i in range(start_index, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        results = []

        for _, row in chunk.iterrows():
            best_country, best_score = compute_best_country(row)
            results.append({
                'Character': row['Character'],
                'Best_Country': best_country,
                'Best_Score': best_score
            })

        # Convert the results into a DataFrame
        chunk_results = pd.DataFrame(results)
        
        # Add the results to the existing DataFrame
        existing_df = pd.concat([existing_df, chunk_results], ignore_index=True)
        
        # Save to the file
        existing_df.to_csv(output_file, index=False)
        print(f"Progress: Processing rows {i} to {i+chunk_size-1}")


# Start the processing
process_in_chunks_with_best_country(df_character_influence, chunk_size=100, output_file='character_countries.csv')
